# Create new column to indicate high value customers

## Import libraries

In [1]:
import dataiku
from dataiku import pandasutils as pdu
import pandas as pd
from myfunctions import create_target
#testing git

## Read dataset
###### Dataiku abstracts data access with the `dataiku.Dataset` class.

###### You use the same API to read the data regardless where the data is stored (PostgreSQL, Snowflake, S3, Azure Blob, GCS etc.)

In [2]:
# Read the dataset as a Pandas dataframe in memory
# Note: here, sampling options are available
dataset_SF_joined = dataiku.Dataset("SF_joined")
df = dataset_SF_joined.get_dataframe()

## Process data in pandas Dataframe

In [3]:
# Get some simple descriptive statistics
pdu.audit(df)

,_a_variable,_b_data_type,_c_cardinality,_d_missings,_e_sample_values
0,customer_id,object,23399,0,"[041b77595d, 6d50fd6b8c]"
1,birth,object,11470,0,"[3/15/1987, 6/29/1967]"
2,price_first_item_purchased,float64,10,0,"[22.0, 44.0]"
3,gender,object,2,0,"[M, F]"
4,revenue,object,448,0,"[171, 162]"
5,ip,object,24799,0,"[108.124.56.200, 172.244.112.177]"
6,Country,object,148,52,"[United States, Taiwan]"
7,pages_visited,float64,14,0,"[2.0, 8.0]"
8,campaign,bool,2,0,"[False, True]"
9,GDP_per_cap,float64,132,133,"[51704.0, 38400.0]"


## Set revenue threshold for high-value customers

In [4]:
# Cast revenue column to float
# This converts 'unknown' values to NaN
# Set threshold for high revenue customer to the median revenue
df['revenue'] = pd.to_numeric(df['revenue'], errors='coerce', downcast='float')
print('Median revenue: ' + str(df['revenue'].median()))


high_rev = 171

df.head(10)

Median revenue: 171.0


,customer_id,birth,price_first_item_purchased,gender,revenue,ip,Country,pages_visited,campaign,GDP_per_cap
0,041b77595d,3/15/1987,22.0,M,171.0,108.124.56.200,United States,2.0,False,51704.0
1,6d50fd6b8c,6/29/1967,22.0,F,162.0,172.244.112.177,United States,8.0,True,51704.0
2,e9ecafc280,5/6/1963,44.0,F,285.0,70.217.142.6,United States,8.0,False,51704.0
3,5d9d084327,6/30/1962,15.5,F,194.0,118.163.133.4,Taiwan,4.0,False,38400.0
4,6239d12d39,2/20/1966,10.0,F,154.0,199.110.47.69,United States,6.0,True,51704.0
5,606aa67189,11/1/1992,22.0,M,NaN,175.225.47.60,South Korea,4.0,False,31950.0
6,d410061099,2/17/1985,10.0,F,225.0,134.8.154.245,United States,8.0,False,51704.0
7,9643746e6f,10/6/1989,28.0,F,79.0,171.208.240.197,China,5.0,True,9055.0
8,d37d18bf79,6/24/1977,15.5,F,120.0,62.173.251.103,United Kingdom,4.0,True,36569.0
9,f71b749617,5/11/1983,42.0,M,215.0,159.48.29.94,United States,5.0,False,51704.0


## Create new column `high_value`

In [5]:
df['high_value'] = df.apply(create_target, axis = 1, v = high_rev)
df.head(10)

,customer_id,birth,price_first_item_purchased,gender,revenue,ip,Country,pages_visited,campaign,GDP_per_cap,high_value
0,041b77595d,3/15/1987,22.0,M,171.0,108.124.56.200,United States,2.0,False,51704.0,1.0
1,6d50fd6b8c,6/29/1967,22.0,F,162.0,172.244.112.177,United States,8.0,True,51704.0,0.0
2,e9ecafc280,5/6/1963,44.0,F,285.0,70.217.142.6,United States,8.0,False,51704.0,1.0
3,5d9d084327,6/30/1962,15.5,F,194.0,118.163.133.4,Taiwan,4.0,False,38400.0,1.0
4,6239d12d39,2/20/1966,10.0,F,154.0,199.110.47.69,United States,6.0,True,51704.0,0.0
5,606aa67189,11/1/1992,22.0,M,NaN,175.225.47.60,South Korea,4.0,False,31950.0,NaN
6,d410061099,2/17/1985,10.0,F,225.0,134.8.154.245,United States,8.0,False,51704.0,1.0
7,9643746e6f,10/6/1989,28.0,F,79.0,171.208.240.197,China,5.0,True,9055.0,0.0
8,d37d18bf79,6/24/1977,15.5,F,120.0,62.173.251.103,United Kingdom,4.0,True,36569.0,0.0
9,f71b749617,5/11/1983,42.0,M,215.0,159.48.29.94,United States,5.0,False,51704.0,1.0
